In [ ]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import optuna
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [16]:
def load_datasets(data_directory = "./data"):
    train_tf = transforms.Compose([
        transforms.ToTensor(), 
        transforms.RandomHorizontalFlip(.2), 
        transforms.RandomVerticalFlip(.1), 
        transforms.RandomRotation(30), 
        transforms.Normalize((0.49139965, 0.48215845, 0.44653094),(0.24703224, 0.24348514, 0.26158786))
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.49139965, 0.48215845, 0.44653094),(0.24703224, 0.24348514, 0.26158786))
    ])


    cifar_train_set = torchvision.datasets.CIFAR10(root = './data', train = True, download = True)
    train_set, valid_set = torch.utils.data.random_split(cifar_train_set,[45000,5000])
    test_set = torchvision.datasets.CIFAR10(root = './data', train = False, download = False, transform = test_tf)
    
    train_set.dataset.transform = train_tf
    valid_set.dataset.transform = test_tf
    
    return train_set, valid_set, test_set

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

AttributeError: 'FrozenTrial' object has no attribute 'step'

In [ ]:
# CNN model

class cifarnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,64,3,1,1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64,128,3,1,1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128,256,3,1,1)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256,512,3,1,1)
        self.bn4 = nn.BatchNorm2d(512)

        self.fc1 = nn.Linear(512*2*2,10)
        #self.fc2 = nn.Linear(128,10)
        #self.fc2 = nn.Linear(128,256)
        #self.fc3 = nn.Linear(256,512)
        #self.fc4 = nn.Linear(512,1024)
        #self.fc5 = nn.Linear(1024,10)

    def forward(self,x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        #print(x.shape)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = torch.flatten(x,1)
        x = self.fc1(x)

        return x

In [ ]:
def objective(trial):
    model = cifarnet()
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    
    lr = trial.suggest_float("lr",1e-5,1e-1,log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_set, valid_set, test_set = load_datasets()
    batch_size=trial.suggest_int("batch_size", 6, 11, step=1)
    
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=2**batch_size,
        shuffle=True,
        num_workers=2)
    val_loader = torch.utils.data.DataLoader(
        valid_set,
        batch_size=2**batch_size,
        shuffle=False,
        num_workers=2)
    
    epochs = 25
    best_val_acc = -np.inf
    for epoch in range(epochs):  # loop over the dataset multiple times
        epoch_steps = 0
        model.train()
        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation loss
        val_loss = 0.0
        total = 0
        correct = 0
        model.eval()
        for i, data in enumerate(val_loader):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.item()

        accuracy = correct/len(val_loader.dataset)
        if accuracy > best_val_acc:
            best_val_acc = accuracy
            # If this model's val acc is best, save/overwrite best model for this trial
            trial.set_user_attr('Epoch',epoch)
            with open("./pickle_models/{}.pickle".format(trial.number), "wb") as fout:
                pickle.dump(model, fout)
            
        trial.report(accuracy,epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
    print("Finished Training")
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
print('Best epoch: {}'.format(trial.user_attrs))
# Load the best model.
with open("./pickle_models/{}.pickle".format(trial.number), "rb") as fin:
    best_model = pickle.load(fin)
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete','duration','number'], axis=1)
print(df)

In [ ]:
# Test network
correct = 0
total = 0

test_loader = DataLoader(test_dataset,batch_size=2**trial.params["batch_size"],shuffle=False)
# no gradients necessary when testing
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        output = best_model(images)
        
        _, predicted = torch.max(output.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on test images: {100 * correct // total} %')